In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import re
import requests
pd.options.display.max_rows = 5000 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# shengxiao = pd.DataFrame({'shu':[5,10,15], 'niu':[100,200,300],'hu':[1,5,10]}, index=['2018', '2019', '2020'])
# shengxiao

In [11]:
shengxiao = pd.DataFrame({'shu':[5,10,15], 'niu':[100,150,200]}, index=[ '2018','2019', '2020'])
shengxiao

,shu,niu
2018,5,100
2019,10,150
2020,15,200


In [12]:
weights = np.array([0.2, 0.8])

In [13]:
return_each = shengxiao.pct_change()
return_each

,shu,niu
2018,NaN,NaN
2019,1.0,0.500000
2020,0.5,0.333333


In [14]:
return_each_mean = return_each.mean() 
return_each_mean # 

shu    0.750000
niu    0.416667
dtype: float64

### 以列为计算方向，感觉每次都会出现一点最终结果上的偏差

allocated_return是平均每次的-----平均回报率

比如，初始投资1000，shu占20%， niu占80%，

1000 * 1.483 = 1483

1483 * 1.483 = 2199 约等于手动计算的2200

In [15]:
allocated_return = np.sum(weights * return_each_mean)
allocated_return

0.48333333333333334

### 以行为计算单位，计算结果比较准确，但每次算的都是整个portfolio的一次回报率

each_portfolio_return 给出每次的------实际回报率

比如，依然是初始投资1000，shu占20%， niu占80%

0.6 = 20%权数 * 100%roi + 80%权数 * 50%roi

1000 * （1+0.6）= 1600     

1600 * （1+0.367）= 2187.2 约等于手动计算的2200


In [16]:
shengxiao['each_portfolio_return'] = return_each.dot(weights)

In [17]:
shengxiao

,shu,niu,each_portfolio_return
2018,5,100,NaN
2019,10,150,0.600000
2020,15,200,0.366667


### cumulative 的正确用法， 感觉之前的code有一些错误

In [18]:
shengxiao['cumulative_return'] = (1+shengxiao['each_portfolio_return']).cumprod()
shengxiao

,shu,niu,each_portfolio_return,cumulative_return
2018,5,100,NaN,NaN
2019,10,150,0.600000,1.600000
2020,15,200,0.366667,2.186667


In [22]:
end_return = (1+shengxiao['each_portfolio_return']).cumprod()[-1] - 1
end_return

1.186666666666667

转化成年化回报率, 3年

In [26]:
end_ann_return = ((1+shengxiao['cumulative_return'][-1])**(1/3) )-1
end_ann_return

0.4715630660748804

In [25]:
shengxiao['cumulative_return'][-1]

2.186666666666667

In [57]:
r1 = [0.047693049976702734,
 0.04622498379079718,
 0.0359501243453683,
 0.020317440981537892,
 0.057176075097929116,
 0.19143280242128802,
 0.04307393950077352,
 0.05932316270221825,
 0.14017590378165554,
 0.15249878435097408,
 0.08807301752569181,
 0.020244273490107396,
 0.09781644203495603]

In [58]:
r2 = [0.03730506023442033,
 0.003443821497637221,
 0.04596146925871816,
 0.07538516381729027,
 0.02516007574832691,
 0.16689490646223817,
 0.07649890109306548,
 0.024061104417790087,
 0.12324767508128423,
 0.14351286457290824,
 0.061402988588690706,
 0.03602490373446339,
 0.18110106549316665]

In [61]:
r3 = [0.061736249923856144,
 0.020332535871040034,
 0.021505822328386087,
 0.08078523218742749,
 0.008627397827318754,
 0.16964739419151295,
 0.09237652118978815,
 0.14956552451222885,
 0.07860970519169834,
 0.17990213293743604,
 0.037855574643010745,
 0.03315501013648788,
 0.06590089905980841]

In [63]:
r4 =[0.027925292715085745,
 0.013343878280026264,
 0.02119749436103972,
 0.13270517769785437,
 0.0010907774013141302,
 0.19501370723039396,
 0.03709082892689144,
 0.0010512164995387686,
 0.21572904021307032,
 0.10155988447850219,
 0.06098620193740988,
 0.003210728756116591,
 0.1890957715027566]

In [64]:
r5 = [0.07755033227568388,
 0.0005224504239839764,
 0.06858322392425512,
 0.013808655350641363,
 0.004314494612770854,
 0.2033257738349047,
 0.11268802130569665,
 0.0040287876568744435,
 0.019908580919928955,
 0.1920374180010294,
 0.16494723447861667,
 0.008523270733795598,
 0.1297617564818182]

In [65]:
col=['AHT',
 'AV',
 'BA',
 'GSK',
 'HSBA',
 'JD',
 'NG',
 'RDSB',
 'RIO',
 'SPX',
 'TSCO',
 'TW',
 'ULVR']

In [66]:
np.array(r1)+np.array(r2)+ np.array(r3)  +np.array(r4)+ np.array(r5)

array([0.25220999, 0.08386767, 0.19319813, 0.32300167, 0.09636882,
       0.92631458, 0.36172821, 0.2380298 , 0.57767091, 0.76951108,
       0.41326502, 0.10115819, 0.66367593])

In [67]:
mean_weight = (np.array(r1)+np.array(r2)+ np.array(r3)  +np.array(r4)+ np.array(r5))/5

In [69]:
pd.DataFrame({'index':col, 'mean_weight':mean_weight}).sort_values(by='mean_weight', ascending=False)

,index,mean_weight
5,JD,0.185263
9,SPX,0.153902
12,ULVR,0.132735
8,RIO,0.115534
10,TSCO,0.082653
6,NG,0.072346
3,GSK,0.064600
0,AHT,0.050442
7,RDSB,0.047606
2,BA,0.038640
